In [ ]:
import streamlit as st
import random
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

# Load JSON with intents, patterns, and responses
intents = {
    "intents": [
        {
            "intent": "check_balance",
            "patterns": [
                "What's my account balance?",
                "How much money do I have?",
                "Check my balance",
                "What is the balance in my account?"
            ],
            "responses": [
                "Your current account balance is $1,500.",
                "You have $1,500 available in your account.",
                "The balance in your account is $1,500."
            ]
        },
        {
            "intent": "transaction_history",
            "patterns": [
                "Show me my transaction history.",
                "What are my recent transactions?",
                "Can I see my last few transactions?",
                "Give me a report of my recent activity."
            ],
            "responses": [
                "Here are your last 5 transactions: \n1. $100 - Grocery Store\n2. $50 - Gas Station\n3. $250 - Rent Payment\n4. $200 - Online Shopping\n5. $75 - Restaurant",
                "Your recent transactions are as follows: \n1. $100 for Grocery Store\n2. $50 for Gas\n3. $250 for Rent\n4. $200 for Shopping\n5. $75 for Dining.",
                "Here’s a summary of your latest transactions: \n1. Grocery Store: $100\n2. Gas Station: $50\n3. Rent Payment: $250\n4. Online Shopping: $200\n5. Restaurant: $75"
            ]
        },
        {
            "intent": "transfer_funds",
            "patterns": [
                "Transfer money to my friend.",
                "Send $500 to my savings account.",
                "Can you transfer $200 to my checking account?",
                "I need to transfer funds."
            ],
            "responses": [
                "Please provide the account number and amount to transfer.",
                "To complete the transfer, please specify the recipient’s account and amount.",
                "I can help with that. Please provide the recipient's details and amount."
            ]
        },
        {
            "intent": "check_account_details",
            "patterns": [
                "What’s the name on my account?",
                "Who is the account holder?",
                "Can you give me my account details?",
                "Show me the details of my account."
            ],
            "responses": [
                "The name on your account is John Doe.",
                "Your account holder’s name is John Doe.",
                "Your account details are: Name - John Doe, Account Type - Checking."
            ]
        },
        {
            "intent": "bill_payment",
            "patterns": [
                "Pay my electricity bill.",
                "I want to pay my phone bill.",
                "Can I pay my utility bill now?",
                "Pay $150 for my electricity bill."
            ],
            "responses": [
                "Please confirm the bill amount and account details to proceed.",
                "I need the bill amount and the recipient’s account number to make the payment.",
                "I can help with the payment. Please provide the bill amount and details."
            ]
        },
         {
            "intent": "check_loan_details",
            "patterns": [
                "What's the status of my loan?",
                "Tell me about my loan.",
                "Do I have any loans?",
                "Give me details of my loan."
            ],
            "responses": [
                "Your current loan balance is $10,000 with an interest rate of 5%.",
                "You have an outstanding loan of $10,000 at 5% interest.",
                "Your loan is at 5% interest, with a balance of $10,000."
            ]
        },
        {
            "intent": "credit_card_inquiry",
            "patterns": [
                "What is my credit card limit?",
                "Tell me my credit card balance.",
                "How much available credit do I have?",
                "What is my credit card statement?"
            ],
            "responses": [
                "Your current credit limit is $5,000, and your available balance is $2,000.",
                "Your credit card limit is $5,000, with $2,000 available.",
                 "Your current balance on the credit card is $2,000 out of your $5,000 limit."
            ]
        },
        {
            "intent": "account_opening",
            "patterns": [
                "How can I open a new account?",
                "I want to open a new account.",
                "Can you help me open a checking account?",
                "I would like to open a new savings account."
            ],
            "responses": [
                "To open a new account, please provide your full name, address, and ID proof.",
                "We can open a new account for you. Please provide the necessary identification details.",
                "Please submit your personal information to begin the account opening process."
            ]
        },
        {
            "intent": "account_closure",
            "patterns": [
                "I want to close my account.",
                "Can you close my checking account?",
                "How do I close my bank account?",
                "I wish to terminate my account."
            ],
             "responses": [
                "To close your account, please visit the nearest branch with your identification.",
                "You need to submit a formal request to close your account at the branch.",
                "Please provide your account details so we can assist you with the account closure."
            ]
        },
        {
            "intent": "request_checkbook",
            "patterns": [
                "I need a new checkbook.",
                "Can you send me a checkbook?",
                "Order a checkbook for me.",
                "Request a new checkbook."
            ],
            "responses": [
                "Please confirm the account type, and we will order your checkbook.",
                "I’ll need the details of your account type to request a new checkbook.",
                "You can request a new checkbook by providing the necessary details."
            ]
        },
        {
            "intent": "set_security_alert",
            "patterns": [
                "Set a security alert for my account.",
                "Enable transaction alerts.",
                "I want to receive alerts for my account.",
                "Set up an SMS alert for transactions."
                ],
            "responses": [
                "Your security alert has been set up. You will receive notifications for all transactions.",
                "Transaction alerts have been enabled for your account.",
                "You will now receive security alerts for any transactions on your account."
            ]
        },
        {
            "intent": "change_password",
            "patterns": [
                "Can I change my account password?",
                "Help me reset my password.",
                "I want to change my password.",
                "How do I change my account password?"
            ],
            "responses": [
                "To change your password, please visit the security section of your online banking.",
                "You can change your password through the security settings in the mobile app.",
                "Please follow the password reset instructions sent to your email."
            ]
        },
        {
            "intent": "update_contact_information",
            "patterns": [
                "I need to update my contact details.",
                "Can I update my phone number?",
                "I want to change my address.",
                "Help me update my contact info."
            ],
             "responses": [
                "Please provide your new contact details, and we’ll update them for you.",
                "To update your contact information, visit the settings section of your account.",
                "You can update your contact details by contacting customer support or via the app."
            ]
        },
        {
            "intent": "loan_payment",
            "patterns": [
                "I want to make a loan payment.",
                "Pay my loan installment.",
                "How can I pay my loan?",
                "Pay $200 towards my loan."
            ],
            "responses": [
                "Please provide the payment amount and method to proceed with the loan payment.",
                "To make a loan payment, let us know the amount and payment details.",
                "I can help with that! What amount would you like to pay towards your loan?"
            ]
        },
        {
            "intent": "locate_atm_or_branch",
            "patterns": [
                "Where is the nearest ATM?",
                "Find me a branch near me.",
                "Where can I withdraw money?",
                "Locate an ATM for me."
            ],
            "responses": [
                "Please provide your location, and I’ll find the nearest ATM for you.",
                "You can visit the nearest branch at 123 Main Street or use our ATM locator app.",
                "Our nearest branch is located at 456 Elm Street. ATM is also available."
            ]
        }
    ]
}


# Preprocess and prepare the data
def preprocess_data(intents):
    all_patterns = []
    all_intents = []
    all_responses = []
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            # Convert to lowercase and remove unnecessary characters (basic cleaning)
            pattern_cleaned = pattern.lower()
            all_patterns.append(pattern_cleaned)
            all_intents.append(intent['intent'])
            all_responses.append(intent['responses'])
    return all_patterns, all_intents, all_responses

# Extract training data
patterns, intents_labels, responses = preprocess_data(intents)

# Create a model pipeline (CountVectorizer + Naive Bayes)
model = make_pipeline(CountVectorizer(), MultinomialNB())

# Train the model
model.fit(patterns, intents_labels)

# Function to get a response based on the intent
def get_response(user_input):
    # Basic input cleaning: Convert to lowercase
    user_input_cleaned = user_input.lower()
    predicted_intent = model.predict([user_input_cleaned])[0]
    intent_idx = intents_labels.index(predicted_intent)
    return random.choice(responses[intent_idx])

# Streamlit app code
def main():
    st.title("Banking Chatbot")
    
    st.write("Welcome! I can assist you with your banking needs.")
    user_input = st.text_input("How can I assist you today?")

    if user_input:
        response = get_response(user_input)
        st.write(f"Bot: {response}")

if __name__ == '__main__':
    main()


In [2]:
!jupyter nbconvert --to script chatbot.ipynb


[NbConvertApp] Converting notebook chatbot.ipynb to script
[NbConvertApp] Writing 11812 bytes to chatbot.py


In [ ]:
!streamlit run chatbot.py